In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [82]:
import os
import shutil

## ChangeImg

In [83]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

In [84]:
from albumentations.pytorch import ToTensorV2

In [93]:
import glob
singal = "55"
data_path = glob.glob('./newdata/'+singal+'/Positive/*.jpg')
data_path.sort()

In [94]:
from PIL import Image
img = Image.open(data_path[1]).convert('RGB')

## demo

In [95]:
import cv2
import matplotlib.pyplot as plt

In [96]:
target = './newdata/'+singal+'/'
img_name = '00.jpg'
img = cv2.imread('./data/Positive/0.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

image1 = Compose([
        # 对比度受限直方图均衡
            #（Contrast Limited Adaptive Histogram Equalization）
        CLAHE(),
        # 随机旋转 90°
        RandomRotate90(),
        # 转置
        Transpose(),
        # 随机仿射变换
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
        # 模糊
        Blur(blur_limit=3),
        # 光学畸变
        OpticalDistortion(),
        # 网格畸变
        GridDistortion(),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
    # 随机改变图片的 HUE、饱和度和值
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    ], p=1.0)(image=img)['image']
# plt.figure(figsize=(10, 10))
# plt.imshow(image1)
image2 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
cv2.imwrite( target+img_name, image2);

In [97]:
def change_img(img_path):
    img = Image.open(img_path).convert('RGB')
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cnt=3
    for c in range(cnt):
        image1 = Compose([
                # 对比度受限直方图均衡
                    #（Contrast Limited Adaptive Histogram Equalization）
                CLAHE(),
                # 随机旋转 90°
                RandomRotate90(),
                # 转置
                Transpose(),
                # 随机仿射变换
                ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
                # 模糊
                Blur(blur_limit=3),
                # 光学畸变
                OpticalDistortion(),
                # 网格畸变
                GridDistortion(),
                # 随机改变图片的 HUE、饱和度和值
                HueSaturationValue()
            ], p=1.0)(image=img)['image']
#         plt.figure(figsize=(10, 10))
#         plt.imshow(image1)
        image2 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

        filepath = img_path.split('\\')
#         print(filepath[0])
        name = filepath[1].split('.')[0]
        houzhui = filepath[1].split('.')[1]
#         print(filepath[0]+'/'+name+'-'+str(c)+'.'+houzhui)
        cv2.imwrite( filepath[0]+'/'+name+'-'+str(c)+'.'+houzhui, image2);
    

In [98]:
'./data/Positive\\0.jpg'.split('\\')

['./data/Positive', '0.jpg']

In [99]:
for i in data_path:
    change_img(i)

In [100]:
import os
import shutil
cnt=0
train_target='./newdata/'+singal+'/train/'
json_path1 = 'train_data_'+singal+'.json'
test_target='./newdata/'+singal+'/test/'
positive = './newdata/'+singal+'/Positive/'
negative = './newdata/'+singal+'/Negative/'
test = "./newdata/"+singal+"/Unverified/"

isExists=os.path.exists(train_target)
if not isExists:
        os.makedirs(train_target) 
train_data_json = {}
for root, dirs, files in os.walk(positive, topdown=False):
    for name in files:
        shutil.copy(os.path.join(root, name), train_target+str(cnt)+'.jpg')
        train_data_json[str(cnt)+'.jpg']=1
        cnt+=1

for root, dirs, files in os.walk(negative, topdown=False):
    for name in files:
        shutil.copy(os.path.join(root, name), train_target+str(cnt)+'.jpg')
        train_data_json[str(cnt)+'.jpg']=0
        cnt+=1      

import json
with open(json_path1, 'w') as fp:
    json.dump(train_data_json, fp)

cnt=0

isExists=os.path.exists(test_target)
if not isExists:
        os.makedirs(test_target) 

for root, dirs, files in os.walk(test, topdown=False):
    for name in files:
        shutil.copy(os.path.join(root, name), test_target+str(cnt)+'.jpg')
        cnt+=1